# Assignment Problem: 0-1 Knapsack

In [1]:
from pyomo.environ import *
import numpy as np

Given a set of $N=6$ items with their *weights* $w_i$ kg and *values* $v_i$ pesos as follows:

|         | Weight | Value |
| ------- | ------ | ----- |
| Item 1  | 5      | 8     |
| Item 2  | 7      | 3     |
| Item 3  | 4      | 6     |
| Item 4  | 3      | 11    |
| Item 5  | 2      | 4     |
| Item 6  | 4      | 5     |


Find a subset of the items with the maximum total value that can fit in a bag with a maximum capacity of $W_{max}=14$.

In [3]:
# Create the model object

def create_Assignment_mdl(N, W, V, Wmax):
    # N    = set of items
    # W    = vector of weights
    # V    = vector of values
    # Wmax = maximum capacity
    
    model = ConcreteModel(name="knapsack")

    # Declare decision variables and parameters
    model.x = Var(N, domain=Binary)
    model.w = Param(N, initialize=lambda mdl, i: W[i-1])
    model.v = Param(N, initialize=lambda mdl, i: V[i-1])
    
    def obj_rule(mdl):
        return sum(mdl.x[i]*mdl.v[i] for i in N)
    
    def capacity_rule(mdl):
        return sum(mdl.x[i]*mdl.w[i] for i in N) <= Wmax
    
    model.obj = Objective(rule=obj_rule, sense=maximize)
    model.cap = Constraint(rule=capacity_rule)
    
    return model

# Specify the data
N = RangeSet(6)
W = [5, 7, 4, 3, 2, 4]
V = [8, 3, 6, 11, 4, 5]
Wmax = 14

# Solve the model
model = create_Assignment_mdl(N, W, V, Wmax)
solver = SolverFactory("glpk") 
res = solver.solve(model)
model.display() 
model.x.pprint()
res.write()

Model knapsack

  Variables:
    x : Size=6, Index=[1:6]
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
          6 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  29.0

  Constraints:
    cap : Size=1
        Key  : Lower : Body : Upper
        None :  None : 14.0 :  14.0
x : Size=6, Index=[1:6]
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.0 :     1 : False : False : Binary
      2 :     0 :   0.0 :     1 : False : False : Binary
      3 :     0 :   1.0 :     1 : False : False : Binary
      4 :     0 :   1.0 :     1 : False : 